# Query Rewrite
- 질문 복잡성이 높을 때
- 앞단에 라우터 걸어서 복잡성 높은 쿼리에 대한 셀렉터 모듈로 활용될 수 있는 구조

In [1]:
!pip install llama_index openai datasets llama-index-retrievers-bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154

In [2]:
# set up OpenAI
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

OpenAI API Key:··········


In [3]:


import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from IPython.display import Markdown, display
import pprint
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small"
)
Settings.llm= OpenAI(model='gpt-4o-mini')

In [4]:
# Dataset 로드
from datasets import load_dataset

ds = load_dataset("HAERAE-HUB/KOREAN-WEBTEXT", split='train[:20]')
data = ds.to_pandas()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1284879 [00:00<?, ? examples/s]

In [5]:
data

,text,source,token_count,__index_level_0__
0,사이트의 판매량에 기반하여 판매량 추이를 반영한 인터파크 도서에서의 독립적인 판매 ...,oscar2201,3348,0
1,“아~아~잊으랴 어찌 우리 이날을 조국의 원수들이 짓밟아 오던 날을~”6·25의 노...,oscar2201,1427,1
2,일러전쟁의 승패를 가른 쓰시마 해전은 세계 최강으로 평가 받는 발틱함대를 괴멸시켰다...,oscar2201,2458,2
3,"재테크 채널 유튜버이자, 「빚부터 갚아라」, 「원트재무설계 소원을 말해봐」 저자인 ...",oscar2201,2838,3
4,"상급자의 범죄와 비리, 부패를 하급자에게 돌리는 것으로 따지면 타의추종을 불허하는 ...",oscar2201,1628,4
5,최근 언론보도에 의하면 이재현 CJ그룹 회장이 지난해 말 두 자녀에게 증여하였던 주...,oscar2201,1366,5
6,"나는 노무현의 시대를 살지 않았다. 그러니까, 나는 이 땅의 생명체로 살아있긴 했지...",oscar2201,2017,6
7,CBRE가 21일 발표한 ‘2021년 2분기 국내 상업용 부동산 시장 보고서’에 따...,oscar2201,1421,7
8,"안녕하세요. 한화솔루션입니다. 지난주, 슬기로운 솔루션 직장생활 2탄에 이어 이번엔...",oscar2201,2143,8
9,캐나다는 3 년 연속 지구상에서 가장 주목할만한 국가로 선포되었습니다. 일반 타이틀...,oscar2201,1104,9


In [6]:
# Document 오브젝트로 변환
from llama_index.core import Document, VectorStoreIndex
docs = []

#Iterative하게 Document 만들기
for i, row in data.iterrows():
    docs.append(Document(
        text=row['text'],
        # extra_info={'title': row['title']}
    ))

In [7]:
index = VectorStoreIndex.from_documents(
    docs
)

## 1. 커스텀 프롬프트 활용

In [8]:
from llama_index.core import PromptTemplate
from llama_index.llms.openai import OpenAI



query_gen_str = """
너는 사용자가 대충 쓴 질문에 대해서, 최대한 답변하기 위한 근거를 찾기 위한 다수의 서치 쿼리를 생성해 내야해.
생성하는 쿼리 중에 딱 하나는 역으로 된 질문으로 추가해서, 상대적 근거로 활용될 수 있도록 해야해.
{num_queries}개의 서치 쿼리를 만들어 내고, 하나당 한줄씩 사용해.
Query: {query}
Queries:
"""

# 프롬프트템플릿화
query_gen_prompt = PromptTemplate(query_gen_str)

#쿼리 브레이커에 사용할 llm 정의
llm = OpenAI(model="gpt-4o-mini")


def generate_queries(llm, query: str, num_queries: int = 4):
    response = llm.predict(
        query_gen_prompt, num_queries=num_queries, query=query
    )
    queries = response.split("\n")
    queries_str = "\n".join(queries)
    print(f"Generated queries:\n{queries_str}")
    return queries

In [9]:
# 잘 답변못했던 질문 확인
queries = generate_queries(llm,"프랭키가 누구한테 인기가 없을까?")

Generated queries:
1. 프랭키의 인기와 관련된 정보
2. 프랭키가 싫어하는 캐릭터나 사람들
3. 프랭키의 팬층 분석
4. 프랭키가 누구에게 인기가 있는가?


In [10]:
queries

['1. 프랭키의 인기와 관련된 정보',
 '2. 프랭키가 싫어하는 캐릭터나 사람들',
 '3. 프랭키의 팬층 분석',
 '4. 프랭키가 누구에게 인기가 있는가?']

In [11]:
from tqdm.asyncio import tqdm
import nest_asyncio
nest_asyncio.apply()

# 다수의 쿼리를 동시에 날려야 하기 떄문에 async 사용
# 사용할 다수 Retriever들의 결과를 합치는 용도
async def run_queries(queries, retrievers):
    tasks = []
    for query in queries:
        for i, retriever in enumerate(retrievers):
            tasks.append(retriever.aretrieve(query))

    task_results = await tqdm.gather(*tasks)

    results_dict = {}
    for i, (query, query_result) in enumerate(zip(queries, task_results)):
        results_dict[(query, i)] = query_result

    return results_dict

In [12]:
!pip install llama-index-retrievers-bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pystemmer: filename=PyStemmer-2.2.0.1-cp310-cp310-linux_x86_64.whl size=579703 sha256=e3fa8f9f8db1373c059b05359663d9fe872774d66b5858ad3bdf165d4c5242ff
  Stored in directory: /root/.cache/pip/wheels/45/7d/2c/a7ebb8319e01acc5306fa1f8558bf24063d6cec2c02de330c9
Successfully built pystemmer


In [13]:

from llama_index.retrievers.bm25 import BM25Retriever


## vector retriever
vector_retriever = index.as_retriever(similarity_top_k=2)

## bm25 retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=2
)

#query_engine = index.as_query_engine()
#response = query_engine.query(query_str)
#display(Markdown(f"{response}"))

DEBUG:bm25s:Building index from IDs objects


Building index from IDs objects


In [14]:
results_dict = await run_queries(queries, [vector_retriever, bm25_retriever])

100%|██████████| 8/8 [00:00<00:00, 13.73it/s]


In [15]:
results_dict

{('1. 프랭키의 인기와 관련된 정보',
  0): [NodeWithScore(node=TextNode(id_='bb6b6e9b-a746-45e5-88e2-132bfae682b5', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='aa1370bd-25d4-4cbe-90a7-4c508cc12f92', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='7185be72186b8e1fcf09a1814f43e77f119a5748a816c7013d097fae051df68d'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='f8fea089-898a-4dca-9719-0b5687b30846', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='afb22e509851281e67bb9a6fa75623623331a63ccd47b4b66fb627293bdaff0b')}, text='레고 부스트를 만난 지 2달 정도 되었는데 생각보다 아이들이 잘 따라 만들고 너무 좋아합니다. 직접 움직임을 줄 수 있다는 게 성취감을 높여주는 것 같더라구요. 레고는 모든 아이들이 좋아하는 브릭이기도 하구요. 또 아이와 함께 만들다 보면 자연스럽게 아이들과 더 교감하게 되기도 하구요. 프랭키는 가상의 고양이이지만 애교도 부리고 먹이도 먹는 등 다양한 교감 활동을 할 수 있는데, 단순히 움직이는 코딩이 아니라 교감능력까지 함께 높여줄 수 있는 것 같아서 좋습니다. 특히나 여자아이들이라면 더 좋아할 것 같네요. 애교 많은 재롱둥이 프랭키와 함께 아이들의 상상력과 창의력을 키워 보세요.', mimetype='

In [16]:
from typing import List
from llama_index.core.schema import NodeWithScore

# 여러 Retriever 메서드 사용했기 때문에 결과 퓨전 필요
# 기본적인 rrf 사용
def fuse_results(results_dict, similarity_top_k: int = 2):
    """Fuse results."""
    k = 60.0
    fused_scores = {}
    text_to_node = {}

    for nodes_with_scores in results_dict.values():
        for rank, node_with_score in enumerate(
            sorted(
                nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True
            )
        ):
            text = node_with_score.node.get_content()
            text_to_node[text] = node_with_score
            if text not in fused_scores:
                fused_scores[text] = 0.0
            fused_scores[text] += 1.0 / (rank + k)

    # fusion 스코어 기반 소팅
    reranked_results = dict(
        sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    )
    reranked_nodes: List[NodeWithScore] = []
    for text, score in reranked_results.items():
        reranked_nodes.append(text_to_node[text])
        reranked_nodes[-1].score = score

    return reranked_nodes[:similarity_top_k]

In [17]:
final_results = fuse_results(results_dict)

In [18]:
for n in final_results:
    print(n.score, "\n", n.text, "\n********\n")

0.03333333333333333 
 레고 부스트를 만난 지 2달 정도 되었는데 생각보다 아이들이 잘 따라 만들고 너무 좋아합니다. 직접 움직임을 줄 수 있다는 게 성취감을 높여주는 것 같더라구요. 레고는 모든 아이들이 좋아하는 브릭이기도 하구요. 또 아이와 함께 만들다 보면 자연스럽게 아이들과 더 교감하게 되기도 하구요. 프랭키는 가상의 고양이이지만 애교도 부리고 먹이도 먹는 등 다양한 교감 활동을 할 수 있는데, 단순히 움직이는 코딩이 아니라 교감능력까지 함께 높여줄 수 있는 것 같아서 좋습니다. 특히나 여자아이들이라면 더 좋아할 것 같네요. 애교 많은 재롱둥이 프랭키와 함께 아이들의 상상력과 창의력을 키워 보세요. 
********

0.016666666666666666 
 그리고 누가 짧은 답변에 관심이 있습니까? 해외 유학 그리고 미국의 고등교육 기관, 대학교의 유학을 꿈꾸고 있는 분들을 위한 최적의 정보 사이트입니다! 저희 Study in the USA는 국제 학생들을 위한 최고의 유학 안내 사이트입니다. 'Study in the USA'는 지난 40년 넘게, 국제 학생들과 교육의 기회를 공유해 오고 있습니다. 우리는 오직 우수한 교육을 제공하는 고등 교육기관, 인증된 어학 연수원, 대학교 및 명성있는 교육기관들과 협업하고 있습니다. 저희 검색 툴을 활용하시면 자신에게 맞는 학교를 쉽게 검색할 수 있습니다. 학비, 위치, 전공별로 검색하여 비교해 볼 수 있습니다. 해외 유학 준비는 매우 신나는 일이지만 때로는 힘든 일이 될 수도 있습니다. 저희 사이트에 수록된 학교 소개 정보, 동영상, 블로그 등을 읽어보고 실제 유학생들의 경험을 확인하며, 유학 준비를 시작하세요. 저희 Study in the USA의 전문가들이 여러분의 유학 여정을 안내해 드릴 것입니다. 
********



In [19]:
from typing import List

from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore
import asyncio

# 퓨전리트리버 클래스 정의
class FusionRetriever(BaseRetriever):

    def __init__(
        self,
        llm,
        retrievers: List[BaseRetriever],
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._retrievers = retrievers
        self._similarity_top_k = similarity_top_k
        self._llm = llm
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        queries = generate_queries(
            self._llm, query_bundle.query_str, num_queries=4
        )
        results = asyncio.run(run_queries(queries, self._retrievers))
        final_results = fuse_results(
            results, similarity_top_k=self._similarity_top_k
        )

        return final_results

In [20]:
from llama_index.core.query_engine import RetrieverQueryEngine

fusion_retriever = FusionRetriever(
    llm, [vector_retriever, bm25_retriever], similarity_top_k=2
)
# 퓨전리트리버를 쿼리엔진으로 사용
query_engine = RetrieverQueryEngine(fusion_retriever)

In [21]:
query_str="프랭키가 누구한테 인기가 없을까?"

In [22]:
#naive
naive_query_engine = index.as_query_engine()
response = naive_query_engine.query(query_str)


In [23]:
print(str(response))

프랭키는 일본 축구에 대한 한국인의 인식과 관련하여 인기가 없을 것으로 보인다. 한국에서 일본 축구에 대한 사실과 다른 객기를 부리는 태도가 언급되며, 이는 프랭키와 같은 인물에게 부정적인 인식을 초래할 수 있다.


In [24]:
response2 = query_engine.query(query_str)

Generated queries:
1. 프랭키의 인기와 관련된 정보
2. 프랭키가 싫어하는 캐릭터나 사람들
3. 프랭키의 팬층 분석
4. 프랭키가 누구에게 인기가 있는가?


100%|██████████| 8/8 [00:00<00:00, 17.66it/s]


In [25]:
print(str(response2))

프랭키는 주로 여자아이들에게 인기가 있을 것으로 보이며, 다른 특정 그룹에 대한 언급은 없습니다. 따라서 인기가 없는 특정 대상을 언급하기는 어렵습니다.
